In [1]:
# Cell 1: Environment Setup and Dependencies
import os
import sys
import platform
import time
from datetime import datetime
from pathlib import Path
import json
import yaml
import shutil
import random

# Deep learning
import torch
from ultralytics import YOLO

# Data analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Progress tracking
from tqdm.notebook import tqdm

# Print Python and environment information
print(f"Python version: {platform.python_version()}")
print(f"Platform: {platform.platform()}")

# Check for CUDA
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    else:
        print("CUDA is not available - training will use CPU")
except ImportError:
    print("PyTorch is not installed - you'll need to install it with pip install torch torchvision")

# Check for other required libraries
required_packages = ['numpy', 'matplotlib', 'pandas', 'ultralytics']
for package in required_packages:
    try:
        module = __import__(package.replace('-', '_'))
        print(f"✅ {package} is installed (version: {module.__version__})")
    except ImportError:
        print(f"❌ {package} is NOT installed - use pip install {package}")
    except AttributeError:
        print(f"✅ {package} is installed (version unknown)")

# Manually set the project root path to ensure accuracy
project_root = "/home/peter/Desktop/TU PHD/WildlifeDetectionSystem"
print(f"\nProject root path: {project_root}")

# Output the current working directory for reference
print(f"Current working directory: {os.getcwd()}")

# Create tracking directory
tracking_dir = os.path.join(project_root, "tracking")
os.makedirs(tracking_dir, exist_ok=True)

print("\nEnvironment setup check complete!")

Python version: 3.12.3
Platform: Linux-6.8.0-58-generic-x86_64-with-glibc2.39
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU device: NVIDIA GeForce RTX 4050 Laptop GPU
Number of GPUs: 1
GPU Memory: 5.76 GB
✅ numpy is installed (version: 2.1.1)
✅ matplotlib is installed (version: 3.10.1)
✅ pandas is installed (version: 2.2.3)
✅ ultralytics is installed (version: 8.3.106)

Project root path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem
Current working directory: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/notebooks/training/Planned_Notebooks_v2

Environment setup check complete!


In [2]:
# Cell 2: Load Configuration from Notebook 1
# Find and load the configuration generated by the data preparation notebook
import os
import json
import yaml
from datetime import datetime

# Manually set the project root path to ensure accuracy - this makes the cell self-contained
project_root = "/home/peter/Desktop/TU PHD/WildlifeDetectionSystem"
print(f"Project root path: {project_root}")

def find_latest_config(config_dir, prefix="notebook_data_"):
    """Find the latest configuration file based on timestamp in filename"""
    config_files = [f for f in os.listdir(config_dir) if f.startswith(prefix) and f.endswith('.json')]
    if not config_files:
        return None
    
    # Sort by timestamp (assuming format notebook_data_YYYYMMDD_HHMM.json)
    latest_config = sorted(config_files, reverse=True)[0]
    return os.path.join(config_dir, latest_config)

# Define paths
config_dir = os.path.join(project_root, "config")
if not os.path.exists(config_dir):
    print(f"❌ Config directory not found: {config_dir}")
    print("Please run notebook 1 (data preparation) first")
else:
    # Try to find the latest config file
    latest_config = find_latest_config(config_dir)
    
    if latest_config and os.path.exists(latest_config):
        print(f"Found configuration from notebook 1: {latest_config}")
        
        # Load configuration
        with open(latest_config, 'r') as f:
            notebook1_config = json.load(f)
        
        # Extract key paths and parameters
        timestamp = notebook1_config["timestamp"]
        class_names = notebook1_config["class_names"]
        taxonomic_groups = notebook1_config["taxonomic_groups"]
        standard_export_path = notebook1_config["standard_export_path"]
        hierarchical_export_path = notebook1_config["hierarchical_export_path"]
        
        print(f"\nLoaded configuration with timestamp: {timestamp}")
        print(f"Number of classes: {len(class_names)}")
        print(f"Number of taxonomic groups: {len(taxonomic_groups)}")
        print(f"Standard dataset: {standard_export_path}")
        print(f"Hierarchical dataset: {hierarchical_export_path}")
        
        # Check if the datasets exist
        if os.path.exists(standard_export_path):
            print(f"✅ Standard YOLO dataset exists")
            
            # Verify data.yaml
            data_yaml_path = os.path.join(standard_export_path, 'data.yaml')
            if os.path.exists(data_yaml_path):
                with open(data_yaml_path, 'r') as f:
                    data_yaml = yaml.safe_load(f)
                print(f"   Classes in data.yaml: {data_yaml.get('nc', 'unknown')}")
            else:
                print(f"❌ data.yaml not found in standard dataset")
        else:
            print(f"❌ Standard YOLO dataset not found: {standard_export_path}")
        
        if os.path.exists(hierarchical_export_path):
            print(f"✅ Hierarchical YOLO dataset exists")
            
            # Verify data.yaml
            hierarchical_yaml_path = os.path.join(hierarchical_export_path, 'data.yaml')
            if os.path.exists(hierarchical_yaml_path):
                with open(hierarchical_yaml_path, 'r') as f:
                    hierarchical_yaml = yaml.safe_load(f)
                print(f"   Groups in data.yaml: {hierarchical_yaml.get('nc', 'unknown')}")
            else:
                print(f"❌ data.yaml not found in hierarchical dataset")
        else:
            print(f"❌ Hierarchical YOLO dataset not found: {hierarchical_export_path}")
    else:
        print(f"❌ Configuration from notebook 1 not found in {config_dir}")
        print("Please run notebook 1 (data preparation) first")

# Define the output paths for this notebook
model_save_dir = os.path.join(project_root, "models", "trained")
reports_dir = os.path.join(project_root, "reports")
timestamp_now = datetime.now().strftime("%Y%m%d_%H%M")

# Generate paths for model outputs
standard_model_name = f"wildlife_detector_{timestamp_now}"
hierarchical_model_name = f"wildlife_detector_hierarchical_{timestamp_now}"

standard_model_path = os.path.join(model_save_dir, standard_model_name)
hierarchical_model_path = os.path.join(model_save_dir, hierarchical_model_name)

print(f"\nOutput paths for trained models:")
print(f"- Standard model: {standard_model_path}")
print(f"- Hierarchical model: {hierarchical_model_path}")

# Save the training configuration for reference and tracking
training_config = {
    "notebook": "02_model_training",
    "timestamp": timestamp_now,
    "input": {
        "config": latest_config,
        "standard_dataset": standard_export_path,
        "hierarchical_dataset": hierarchical_export_path,
        "class_names": class_names,
        "taxonomic_groups": taxonomic_groups
    },
    "output": {
        "standard_model": standard_model_path,
        "hierarchical_model": hierarchical_model_path,
        "reports_dir": reports_dir
    }
}

# Save configuration
training_config_path = os.path.join(config_dir, f"training_config_{timestamp_now}.json")
with open(training_config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"\nTraining configuration saved to: {training_config_path}")

Project root path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem
Found configuration from notebook 1: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/config/notebook_data_20250510_0038.json

Loaded configuration with timestamp: 20250510_0038
Number of classes: 30
Number of taxonomic groups: 5
Standard dataset: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038
Hierarchical dataset: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038
✅ Standard YOLO dataset exists
   Classes in data.yaml: 30
✅ Hierarchical YOLO dataset exists
   Groups in data.yaml: 5

Output paths for trained models:
- Standard model: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_1706
- Hierarchical model: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_1706

Training configuration saved to: /home/peter/Desktop/T

In [3]:
# Cell 3: Hardware-Aware Model Selection and Optimization
# Automatically selects the optimal model size based on available hardware

import torch 

def detect_hardware_capabilities():
    """Detect hardware capabilities and recommend model size"""
    # Check for CUDA availability
    cuda_available = torch.cuda.is_available()
    
    if not cuda_available:
        print("No CUDA detected. Using CPU for training.")
        return {
            "device": "cpu",
            "recommended_model": "n",  # nano model for CPU
            "batch_size": 1,
            "image_size": 320,
            "workers": 0
        }
    
    # Get GPU memory in GB
    try:
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        print(f"GPU detected with {gpu_memory:.2f} GB memory")
        
        # Determine model size based on available GPU memory
        if gpu_memory >= 24:
            # High-end GPU (RTX 3090/4090, A100, etc.)
            return {
                "device": 0,
                "recommended_model": "x",  # largest model
                "batch_size": 16,
                "image_size": 640,
                "workers": 4
            }
        elif gpu_memory >= 16:
            # Good GPU (RTX 3080, RTX A6000, etc.)
            return {
                "device": 0,
                "recommended_model": "l",  # large model
                "batch_size": 12,
                "image_size": 640,
                "workers": 4
            }
        elif gpu_memory >= 8:
            # Mid-range GPU (RTX 3070, RTX 2080, etc.)
            return {
                "device": 0,
                "recommended_model": "m",  # medium model
                "batch_size": 8,
                "image_size": 512,
                "workers": 2
            }
        elif gpu_memory >= 4:
            # Entry-level GPU (GTX 1660, RTX 3050, etc.)
            return {
                "device": 0,
                "recommended_model": "s",  # small model
                "batch_size": 4,
                "image_size": 416,
                "workers": 2
            }
        else:
            # Low-end GPU or integrated (MX series, etc.)
            return {
                "device": 0,
                "recommended_model": "n",  # nano model
                "batch_size": 2,
                "image_size": 320,
                "workers": 1
            }
    except Exception as e:
        print(f"Error detecting GPU properties: {e}")
        print("Defaulting to safe configuration")
        return {
            "device": 0 if cuda_available else "cpu",
            "recommended_model": "n",  # nano model for safety
            "batch_size": 1,
            "image_size": 320,
            "workers": 0
        }

# Detect hardware capabilities
print("Detecting hardware capabilities...")
hw_config = detect_hardware_capabilities()

# Map model size to readable name and parameters
model_sizes = {
    "n": {"name": "YOLOv8n", "description": "Nano", "params": "3.2M"},
    "s": {"name": "YOLOv8s", "description": "Small", "params": "11.2M"},
    "m": {"name": "YOLOv8m", "description": "Medium", "params": "25.9M"},
    "l": {"name": "YOLOv8l", "description": "Large", "params": "43.7M"},
    "x": {"name": "YOLOv8x", "description": "Extra Large", "params": "68.2M"}
}

# Display recommended configuration
recommended_model_size = hw_config["recommended_model"]
recommended_model = model_sizes[recommended_model_size]

print("\nRecommended Configuration Based on Hardware:")
print(f"- Model: {recommended_model['name']} ({recommended_model['description']})")
print(f"- Parameters: {recommended_model['params']}")
print(f"- Device: {'GPU' if hw_config['device'] == 0 else 'CPU'}")
print(f"- Batch Size: {hw_config['batch_size']}")
print(f"- Image Size: {hw_config['image_size']}px")
print(f"- Workers: {hw_config['workers']}")

# Allow manual override
print("\nWould you like to override the recommended model size? (y/n)")
override = input().strip().lower()
if override == 'y':
    print("Select model size (n=nano, s=small, m=medium, l=large, x=extra-large):")
    model_input = input().strip().lower()
    if model_input in model_sizes:
        recommended_model_size = model_input
        recommended_model = model_sizes[recommended_model_size]
        print(f"Using {recommended_model['name']} ({recommended_model['description']}) with {recommended_model['params']} parameters.")
    else:
        print(f"Invalid selection. Using recommended {recommended_model['name']}.")

# Define model paths
base_model_path = f"yolov8{recommended_model_size}.pt"
print(f"\nBase model path: {base_model_path}")

# Save hardware configuration to the training config
training_config["hardware"] = hw_config
training_config["model"] = {
    "size": recommended_model_size,
    "name": recommended_model["name"],
    "description": recommended_model["description"],
    "parameters": recommended_model["params"],
    "base_model_path": base_model_path
}

# Update training config file
with open(training_config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"Updated training configuration with hardware and model settings.")

Detecting hardware capabilities...
GPU detected with 5.76 GB memory

Recommended Configuration Based on Hardware:
- Model: YOLOv8s (Small)
- Parameters: 11.2M
- Device: GPU
- Batch Size: 4
- Image Size: 416px
- Workers: 2

Would you like to override the recommended model size? (y/n)


 n



Base model path: yolov8s.pt
Updated training configuration with hardware and model settings.



Base model path: yolov8s.pt
Updated training configuration with hardware and model settings.


In [4]:
# Cell 4: Training Configuration Setup
# Define training parameters for both standard and hierarchical models

# Memory optimization settings
memory_optimizations = {
    "cpu": {
        # CPU-specific optimizations
        "device": "cpu",
        "workers": 0,
        "batch": 1,
        "cache": "disk",
        "imgsz": min(hw_config["image_size"], 320),  # Use 'imgsz' consistently
        "amp": False  # No mixed precision on CPU
    },
    "gpu_low_memory": {
        # For GPUs with less than 4GB memory
        "device": 0,
        "workers": hw_config["workers"],
        "batch": max(1, hw_config["batch_size"] // 2),  # Reduce batch size
        "cache": "disk",
        "imgsz": min(hw_config["image_size"], 384),  # Use 'imgsz' consistently
        "amp": True  # Mixed precision
    },
    "gpu_standard": {
        # For standard GPUs with sufficient memory
        "device": 0,
        "workers": hw_config["workers"],
        "batch": hw_config["batch_size"],
        "cache": "ram",
        "imgsz": hw_config["image_size"],  # Use 'imgsz' consistently
        "amp": True  # Mixed precision
    }
}

# Select memory optimization profile based on hardware
if hw_config["device"] == "cpu":
    memory_profile = "cpu"
elif hw_config["device"] == 0 and torch.cuda.get_device_properties(0).total_memory / (1024**3) < 4:
    memory_profile = "gpu_low_memory"
else:
    memory_profile = "gpu_standard"

print(f"Selected memory optimization profile: {memory_profile}")
memory_config = memory_optimizations[memory_profile]

# Base hyperparameters common to both standard and hierarchical training
base_hyperparams = {
    # Standard YOLOv8 parameters
    'epochs': 100,                    # Maximum number of epochs
    'patience': 25,                   # Early stopping patience
    'optimizer': 'AdamW',             # Optimizer (AdamW better for imbalanced data)
    'lr0': 0.001,                     # Initial learning rate
    'lrf': 0.01,                      # Final learning rate as a fraction of lr0
    'momentum': 0.937,                # SGD momentum/Adam beta1
    'weight_decay': 0.0005,           # Regularization 
    'warmup_epochs': 5,               # Warmup epochs
    'warmup_momentum': 0.8,           # Initial warmup momentum
    'warmup_bias_lr': 0.1,            # Initial warmup learning rate for bias
    
    # Loss function weights
    'box': 7.5,                       # Box loss weight
    'cls': 3.0,                       # Class loss weight
    'dfl': 1.5,                       # DFL loss weight
    
    # Data augmentation
    'hsv_h': 0.015,                   # HSV Hue augmentation
    'hsv_s': 0.7,                     # HSV Saturation augmentation (higher for wildlife)
    'hsv_v': 0.4,                     # HSV Value augmentation (for varying lighting)
    'degrees': 10.0,                  # Rotation augmentation
    'translate': 0.2,                 # Translation augmentation
    'scale': 0.6,                     # Scale augmentation (stronger for wildlife)
    'fliplr': 0.5,                    # Horizontal flip probability
    'mosaic': 1.0,                    # Mosaic augmentation
    'mixup': 0.1,                     # Mixup augmentation
    'copy_paste': 0.1,                # Copy-paste augmentation (for rare classes)
    
    # Saving and checkpointing
    'save': True,                     # Save model
    'save_period': 10,                # Save checkpoints every X epochs
    
    # Nominal batch size for gradient accumulation 
    'nbs': 16                         # Nominal batch size
}

# Merge base hyperparameters with memory optimizations
standard_hyperparams = {**base_hyperparams, **memory_config}
hierarchical_hyperparams = {**base_hyperparams, **memory_config}

# Special adjustments for hierarchical model (fewer classes, may need different parameters)
hierarchical_hyperparams.update({
    'cls': 2.0,                       # Reduced class weight (fewer classes)
    'epochs': 50,                     # Fewer epochs may be sufficient for taxonomic groups
    'patience': 15                    # Earlier stopping 
})

# Display final training configurations
print("\nStandard Model Training Configuration:")
for key, value in standard_hyperparams.items():
    if key in ['epochs', 'patience', 'optimizer', 'lr0', 'batch', 'imgsz', 'device', 'workers', 'amp']:
        print(f"- {key}: {value}")

print("\nHierarchical Model Training Configuration:")
for key, value in hierarchical_hyperparams.items():
    if key in ['epochs', 'patience', 'optimizer', 'lr0', 'batch', 'imgsz', 'device', 'workers', 'amp']:
        print(f"- {key}: {value}")

# Update training config with hyperparameters
training_config["hyperparameters"] = {
    "standard": standard_hyperparams,
    "hierarchical": hierarchical_hyperparams,
    "memory_profile": memory_profile
}

# Update training config file
with open(training_config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"\nHyperparameters added to training configuration.")

Selected memory optimization profile: gpu_standard

Standard Model Training Configuration:
- epochs: 100
- patience: 25
- optimizer: AdamW
- lr0: 0.001
- device: 0
- workers: 2
- batch: 4
- imgsz: 416
- amp: True

Hierarchical Model Training Configuration:
- epochs: 50
- patience: 15
- optimizer: AdamW
- lr0: 0.001
- device: 0
- workers: 2
- batch: 4
- imgsz: 416
- amp: True

Hyperparameters added to training configuration.


In [5]:
# Cell 5: Standard Model Training
# Train the standard model with all species classes

from ultralytics import YOLO 
import time
import numpy as np

# Function to handle out-of-memory errors during training
def train_with_fallback(model, hyperparams, fallbacks=3):
    """Train with automatic fallback to lower resource config if OOM errors occur"""
    for attempt in range(fallbacks + 1):
        try:
            print(f"\nTraining attempt {attempt + 1}/{fallbacks + 1}")
            # Start training timer
            start_time = time.time()
            
            # Train the model with current hyperparams
            results = model.train(**hyperparams)
            
            # If we get here, training was successful
            training_time = time.time() - start_time
            hours, remainder = divmod(training_time, 3600)
            minutes, seconds = divmod(remainder, 60)
            
            print(f"\nTraining completed in {int(hours)}h {int(minutes)}m {int(seconds)}s")
            return results, hyperparams
            
        except RuntimeError as e:
            if 'out of memory' in str(e).lower() and attempt < fallbacks:
                print("\n⚠️ GPU OUT OF MEMORY ERROR DETECTED ⚠️")
                print("Reducing resource usage and trying again...")
                
                # Reduce resource usage
                if hyperparams['batch'] > 1:
                    hyperparams['batch'] = hyperparams['batch'] // 2
                    print(f"Reduced batch size to {hyperparams['batch']}")
                
                if hyperparams['imgsz'] > 320:
                    hyperparams['imgsz'] = max(320, hyperparams['imgsz'] - 64)
                    print(f"Reduced image size to {hyperparams['imgsz']}")
                
                if hyperparams['device'] != 'cpu' and attempt == fallbacks - 1:
                    print("Switching to CPU as last resort")
                    hyperparams['device'] = 'cpu'
                    hyperparams['workers'] = 0
                    hyperparams['amp'] = False
                
                # Free up GPU memory
                torch.cuda.empty_cache()
                import gc
                gc.collect()
                
            else:
                print(f"\nTraining error: {e}")
                return None, hyperparams
    
    return None, hyperparams

# Create model output directories
os.makedirs(os.path.join(model_save_dir), exist_ok=True)
os.makedirs(standard_model_path, exist_ok=True)

print(f"Starting standard model training on all {len(class_names)} classes")
print(f"Using base model: {base_model_path}")
print(f"Dataset path: {standard_export_path}")
print(f"Model will be saved to: {standard_model_path}")

try:
    # Initialize YOLOv8 model
    model = YOLO(base_model_path)
    
    # Set training parameters
    standard_params = {
        **standard_hyperparams,
        'data': os.path.join(standard_export_path, 'data.yaml'),
        'project': model_save_dir,
        'name': os.path.basename(standard_model_path)
    }
    
    print("\nStarting training with the following settings:")
    print(f"- Model: {base_model_path}")
    print(f"- Epochs: {standard_params['epochs']}")
    print(f"- Batch size: {standard_params['batch']}")
    print(f"- Image size: {standard_params['imgsz']}px")
    print(f"- Device: {'CPU' if standard_params['device'] == 'cpu' else 'GPU'}")
    print(f"- Workers: {standard_params['workers']}")
    
    # Train with automatic fallback on OOM errors
    standard_results, final_params = train_with_fallback(model, standard_params)
    
    if standard_results:
        # Get the best epoch (safely)
        if hasattr(standard_results, 'best_epoch'):
            best_epoch = standard_results.best_epoch
        else:
            # Try to infer best epoch from the early stopping message
            import re
            log_output = str(standard_results)
            best_epoch_match = re.search(r'Best results observed at epoch (\d+)', log_output)
            best_epoch = int(best_epoch_match.group(1)) if best_epoch_match else 0
        
        # Get maps safely and convert to Python lists (not NumPy arrays)
        if hasattr(standard_results, 'maps'):
            maps = [float(m) for m in standard_results.maps]
        else:
            # Get from the box attribute
            maps = [float(standard_results.box.map), float(standard_results.box.map50)] if hasattr(standard_results, 'box') else [0, 0]
        
        # Check for actual output directory path
        actual_model_path = standard_model_path
        for dir_name in os.listdir(model_save_dir):
            if os.path.basename(standard_model_path) in dir_name:
                actual_model_path = os.path.join(model_save_dir, dir_name)
                if actual_model_path != standard_model_path:
                    print(f"Note: Model saved with modified path: {actual_model_path}")
        
        # Save final hyperparameters actually used
        training_config["standard_model"] = {
            "train_results": {
                "best_epoch": best_epoch,
                "maps": maps,  # Now using Python list instead of NumPy array
                "fitness": float(getattr(standard_results, 'fitness', 0))
            },
            "final_hyperparams": final_params,
            "actual_model_path": actual_model_path
        }
        
        # Update training config with results
        with open(training_config_path, 'w') as f:
            json.dump(training_config, f, indent=2)
        
        print("\nStandard model training results:")
        print(f"- Best mAP50-95: {maps[0]:.4f}")
        print(f"- Best mAP50: {maps[1]:.4f}")
        print(f"- Best epoch: {best_epoch}")
        print(f"- Model saved to: {actual_model_path}")
        
        # Create a training summary report
        summary_path = os.path.join(reports_dir, f"standard_model_summary_{timestamp_now}.md")
        os.makedirs(os.path.dirname(summary_path), exist_ok=True)
        
        with open(summary_path, 'w') as f:
            f.write(f"# Standard Wildlife Detection Model Training Summary\n\n")
            f.write(f"## Training Metadata\n")
            f.write(f"- **Date and Time**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"- **Model**: {base_model_path}\n")
            f.write(f"- **Dataset**: {standard_export_path}\n")
            f.write(f"- **Classes**: {len(class_names)}\n\n")
            
            f.write(f"## Training Configuration\n")
            for param in ['epochs', 'batch', 'imgsz', 'device', 'optimizer', 'lr0']:
                f.write(f"- **{param}**: {final_params.get(param, 'N/A')}\n")
            
            f.write(f"\n## Performance Metrics\n")
            f.write(f"- **Best mAP50-95**: {maps[0]:.4f}\n")
            f.write(f"- **Best mAP50**: {maps[1]:.4f}\n")
            f.write(f"- **Best epoch**: {best_epoch}\n")

        print(f"Training summary saved to: {summary_path}")
        
        # Save model path for future notebooks
        standard_best_model_path = os.path.join(actual_model_path, "weights", "best.pt")
        training_config["standard_best_model_path"] = standard_best_model_path
        training_config["standard_actual_model_path"] = actual_model_path
        
        # Update training config with paths to result files
        with open(training_config_path, 'w') as f:
            json.dump(training_config, f, indent=2)
    else:
        print("\nStandard model training failed.")

except Exception as e:
    print(f"Error during standard model training: {e}")
    import traceback
    traceback.print_exc()

Starting standard model training on all 30 classes
Using base model: yolov8s.pt
Dataset path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038
Model will be saved to: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_1706

Starting training with the following settings:
- Model: yolov8s.pt
- Epochs: 100
- Batch size: 4
- Image size: 416px
- Device: GPU
- Workers: 2

Training attempt 1/4
New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.106 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 5898MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038/data.yaml, epochs=100, time=None, patience=25, batch=4, imgsz=416, save=True, save_period=10, cache=ram, device=0, workers=2, project=/home/peter/Desk

train: Scanning /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038/labels/train.cache... 356 images, 0 backgrounds, 6 corrupt: 100%|██████████| 356/356 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038/images/train/0092_11_09_100BMCIM_IMAG0004.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0016239]
train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038/images/train/0870_x_IMAG0009.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0185256]
train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038/images/train/1099_21_05_IMAG0136.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0162761]
train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_export_test_01_20250510_0038/images/train/1102_21_05_IMAG0134.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0192918]
train: WARNING ⚠️ /home/peter/Desktop/T

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 350/350 [00:03<00:00, 93.78it/s] 


...

     97/100      1.35G      1.276       5.17      1.415          2        416: 100%|██████████| 88/88 [00:03<00:00, 23.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 38.93it/s]

                   all         86         88       0.82      0.535      0.624      0.387
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 72, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



97 epochs completed in 0.108 hours.
Optimizer stripped from /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_17062/weights/last.pt, 22.5MB
Optimizer stripped from /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_17062/weights/best.pt, 22.5MB

Validating /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_17062/weights/best.pt...
Ultralytics 8.3.106 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 5898MiB)
Model summary (fused): 72 layers, 11,137,194 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 19.07it/s]


                   all         86         88      0.724       0.54      0.653      0.409
         Male Roe Deer         23         23      0.504      0.565      0.619      0.365
       Female Roe Deer         14         14      0.279      0.643      0.343      0.207
                   Fox          8          8       0.71        0.5      0.618      0.367
                Jackal          4          4          1      0.726      0.845      0.615
                Weasel          1          1          1          0      0.995      0.697
               Wildcat          1          1          1          0          0          0
                Rabbit         26         27      0.654      0.889      0.874       0.49
                 Human          9         10      0.645          1      0.931      0.532
Speed: 0.1ms preprocess, 4.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_17062



In [6]:
# Cell 6: Hierarchical Model Training
# Train the hierarchical model with taxonomic groups as classes

# Create model output directories
os.makedirs(hierarchical_model_path, exist_ok=True)

print(f"\nStarting hierarchical model training on {len(taxonomic_groups)} taxonomic groups")
print(f"Using base model: {base_model_path}")
print(f"Dataset path: {hierarchical_export_path}")
print(f"Model will be saved to: {hierarchical_model_path}")

try:
    # Initialize YOLOv8 model
    hierarchical_model = YOLO(base_model_path)
    
    # Set training parameters
    hierarchical_params = {
        **hierarchical_hyperparams,
        'data': os.path.join(hierarchical_export_path, 'data.yaml'),
        'project': model_save_dir,
        'name': os.path.basename(hierarchical_model_path)
    }
    
    print("\nStarting hierarchical training with the following settings:")
    print(f"- Model: {base_model_path}")
    print(f"- Epochs: {hierarchical_params['epochs']}")
    print(f"- Batch size: {hierarchical_params['batch']}")
    print(f"- Image size: {hierarchical_params['imgsz']}px")  # Using 'imgsz' instead of 'image_size'
    print(f"- Device: {'CPU' if hierarchical_params['device'] == 'cpu' else 'GPU'}")
    print(f"- Workers: {hierarchical_params['workers']}")
    
    # Train with automatic fallback on OOM errors
    hierarchical_results, final_hierarchical_params = train_with_fallback(hierarchical_model, hierarchical_params)
    
    if hierarchical_results:
        # Get the best epoch (safely)
        if hasattr(hierarchical_results, 'best_epoch'):
            best_epoch = hierarchical_results.best_epoch
        else:
            # Try to infer best epoch from the early stopping message
            import re
            log_output = str(hierarchical_results)
            best_epoch_match = re.search(r'Best results observed at epoch (\d+)', log_output)
            best_epoch = int(best_epoch_match.group(1)) if best_epoch_match else 0
        
        # Get maps safely and convert to Python lists (not NumPy arrays)
        if hasattr(hierarchical_results, 'maps'):
            maps = [float(m) for m in hierarchical_results.maps]
        else:
            # Get from the box attribute
            maps = [float(hierarchical_results.box.map), float(hierarchical_results.box.map50)] if hasattr(hierarchical_results, 'box') else [0, 0]
        
        # Check for actual output directory path
        actual_model_path = hierarchical_model_path
        for dir_name in os.listdir(model_save_dir):
            if os.path.basename(hierarchical_model_path) in dir_name:
                actual_model_path = os.path.join(model_save_dir, dir_name)
                if actual_model_path != hierarchical_model_path:
                    print(f"Note: Model saved with modified path: {actual_model_path}")
        
        # Save final hyperparameters actually used
        training_config["hierarchical_model"] = {
            "train_results": {
                "best_epoch": best_epoch,
                "maps": maps,  # Now using Python list instead of NumPy array
                "fitness": float(getattr(hierarchical_results, 'fitness', 0))
            },
            "final_hyperparams": final_hierarchical_params,
            "actual_model_path": actual_model_path
        }
        
        # Update training config with results
        with open(training_config_path, 'w') as f:
            json.dump(training_config, f, indent=2)
        
        print("\nHierarchical model training results:")
        print(f"- Best mAP50-95: {maps[0]:.4f}")
        print(f"- Best mAP50: {maps[1]:.4f}")
        print(f"- Best epoch: {best_epoch}")
        print(f"- Model saved to: {actual_model_path}")
        
        # Create a training summary report
        hierarchical_summary_path = os.path.join(reports_dir, f"hierarchical_model_summary_{timestamp_now}.md")
        os.makedirs(os.path.dirname(hierarchical_summary_path), exist_ok=True)
        
        with open(hierarchical_summary_path, 'w') as f:
            f.write(f"# Hierarchical Wildlife Detection Model Training Summary\n\n")
            f.write(f"## Training Metadata\n")
            f.write(f"- **Date and Time**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"- **Model**: {base_model_path}\n")
            f.write(f"- **Dataset**: {hierarchical_export_path}\n")
            f.write(f"- **Taxonomic Groups**: {len(taxonomic_groups)}\n\n")
            
            f.write(f"## Training Configuration\n")
            for param in ['epochs', 'batch', 'imgsz', 'device', 'optimizer', 'lr0']:
                f.write(f"- **{param}**: {final_hierarchical_params.get(param, 'N/A')}\n")
            
            f.write(f"\n## Performance Metrics\n")
            f.write(f"- **Best mAP50-95**: {maps[0]:.4f}\n")
            f.write(f"- **Best mAP50**: {maps[1]:.4f}\n")
            f.write(f"- **Best epoch**: {best_epoch}\n")
            
            f.write(f"\n## Taxonomic Groups\n")
            for group, class_ids in taxonomic_groups.items():
                species = [class_names[idx] for idx in class_ids if idx < len(class_names)]
                f.write(f"- **{group}**: {', '.join(species[:5])}")
                if len(species) > 5:
                    f.write(f" and {len(species)-5} more")
                f.write(f" ({len(species)} species)\n")

        print(f"Hierarchical training summary saved to: {hierarchical_summary_path}")
        
        # Save model path for future notebooks
        hierarchical_best_model_path = os.path.join(actual_model_path, "weights", "best.pt")
        training_config["hierarchical_best_model_path"] = hierarchical_best_model_path
        
        # Update training config with paths to result files
        with open(training_config_path, 'w') as f:
            json.dump(training_config, f, indent=2)
    else:
        print("\nHierarchical model training failed.")

except Exception as e:
    print(f"Error during hierarchical model training: {e}")
    import traceback
    traceback.print_exc()


Starting hierarchical model training on 5 taxonomic groups
Using base model: yolov8s.pt
Dataset path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038
Model will be saved to: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_1706

Starting hierarchical training with the following settings:
- Model: yolov8s.pt
- Epochs: 50
- Batch size: 4
- Image size: 416px
- Device: GPU
- Workers: 2

Training attempt 1/4
New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.106 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 5898MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038/data.yaml, epochs=50, time=None, patience=15, batch=4, imgsz=416, save=True, save_period=10, cache=ram, 

train: Scanning /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038/labels/train.cache... 356 images, 0 backgrounds, 6 corrupt: 100%|██████████| 356/356 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038/images/train/0092_11_09_100BMCIM_IMAG0004.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0016239]
train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038/images/train/0870_x_IMAG0009.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0185256]
train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038/images/train/1099_21_05_IMAG0136.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0162761]
train: WARNING ⚠️ /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/data/export/yolo_hierarchical_test_01_20250510_0038/images/train/1102_21_05_IMAG0134.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0192918]
train: WARNING 

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 350/350 [00:04<00:00, 80.64it/s] 

...

      50/50      1.57G      1.246      2.932      1.364          2        416: 100%|██████████| 88/88 [00:03<00:00, 23.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 39.35it/s]

                   all         86         88      0.873      0.769      0.891      0.548



50 epochs completed in 0.060 hours.
Optimizer stripped from /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_17062/weights/last.pt, 22.5MB
Optimizer stripped from /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_17062/weights/best.pt, 22.5MB

Validating /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_17062/weights/best.pt...
Ultralytics 8.3.106 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 5898MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 31.76it/s]


                   all         86         88       0.91      0.771      0.895      0.563
                  Deer         37         37      0.942      0.874      0.972      0.629
            Carnivores         14         14      0.895      0.643      0.773      0.536
         Small_Mammals         26         27      0.874      0.768      0.885      0.509
                 Other          9         10       0.93        0.8      0.949      0.579
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_17062

Training completed in 0h 3m 48s
Note: Model saved with modified path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_17062

Hierarchical model training results:
- Best mAP50-95: 0.6290
- Best mAP50: 0.5364
- Best epoch: 0
- Model saved to: /home/peter/Desktop/TU PHD/WildlifeDetection

In [7]:
# Cell 7: Training Visualization
# Visualize and compare training results

def make_json_serializable(obj):
    """Convert NumPy types to Python types for JSON serialization"""
    import numpy as np
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_json_serializable(i) for i in obj]
    else:
        return obj

def load_results_csv(model_path, fallback_paths=None):
    """Load and parse the results.csv file from model training with fallback paths"""
    # Try the main path first
    results_path = os.path.join(model_path, 'results.csv')
    if os.path.exists(results_path):
        try:
            print(f"Loading results from: {results_path}")
            return pd.read_csv(results_path)
        except Exception as e:
            print(f"Error loading results from {results_path}: {e}")
    
    # If not found, check if there's a similar path (with suffix)
    if fallback_paths is None:
        base_name = os.path.basename(model_path)
        fallback_paths = []
        if os.path.exists(model_save_dir):
            for dir_name in os.listdir(model_save_dir):
                if base_name in dir_name and dir_name != base_name:
                    fallback_paths.append(os.path.join(model_save_dir, dir_name))
    
    # Try each fallback path
    for path in fallback_paths:
        fallback_results_path = os.path.join(path, 'results.csv')
        if os.path.exists(fallback_results_path):
            try:
                print(f"Loading results from fallback path: {fallback_results_path}")
                return pd.read_csv(fallback_results_path)
            except Exception as e:
                print(f"Error loading results from {fallback_results_path}: {e}")
    
    print(f"No results.csv found for {model_path} or fallbacks")
    return None

def plot_training_metrics(standard_df, hierarchical_df=None, metrics=None):
    """Plot training metrics from results dataframes"""
    if metrics is None:
        # Default metrics to plot (handle different column names in YOLOv8)
        metrics = []
        if standard_df is not None:
            cols = standard_df.columns
            for metric, patterns in {
                'loss': ['train/box_loss', 'box_loss', 'loss'],
                'precision': ['metrics/precision(B)', 'precision', 'val/precision'],
                'recall': ['metrics/recall(B)', 'recall', 'val/recall'],
                'mAP50': ['metrics/mAP50(B)', 'mAP50', 'val/mAP50'],
                'mAP50-95': ['metrics/mAP50-95(B)', 'mAP50-95', 'val/mAP50-95'] 
            }.items():
                # Find the first matching column pattern
                matching_cols = [col for col in cols if any(pattern in col for pattern in patterns)]
                if matching_cols:
                    metrics.append((metric, matching_cols[0]))
    
    # Create subplots for each metric
    if not metrics:
        print("No metrics found in results files")
        return
    
    n_metrics = len(metrics)
    fig, axs = plt.subplots(n_metrics, 1, figsize=(12, 4 * n_metrics))
    if n_metrics == 1:
        axs = [axs]
    
    # Plot each metric
    for i, (metric_name, col_name) in enumerate(metrics):
        ax = axs[i]
        
        # Plot standard model
        if standard_df is not None and col_name in standard_df.columns:
            ax.plot(standard_df['epoch'], standard_df[col_name], 'b-', label='Standard')
            # Mark best epoch
            if 'standard_model' in training_config and 'train_results' in training_config['standard_model']:
                best_epoch = training_config['standard_model']['train_results'].get('best_epoch', 0)
                if best_epoch < len(standard_df):
                    best_value = standard_df.iloc[best_epoch][col_name]
                    ax.plot(best_epoch, best_value, 'bo', markersize=8)
                    ax.axvline(x=best_epoch, color='b', linestyle='--', alpha=0.3)
        
        # Plot hierarchical model if available
        if hierarchical_df is not None and col_name in hierarchical_df.columns:
            ax.plot(hierarchical_df['epoch'], hierarchical_df[col_name], 'r-', label='Hierarchical')
            # Mark best epoch
            if 'hierarchical_model' in training_config and 'train_results' in training_config['hierarchical_model']:
                best_epoch = training_config['hierarchical_model']['train_results'].get('best_epoch', 0)
                if best_epoch < len(hierarchical_df):
                    best_value = hierarchical_df.iloc[best_epoch][col_name]
                    ax.plot(best_epoch, best_value, 'ro', markersize=8)
                    ax.axvline(x=best_epoch, color='r', linestyle='--', alpha=0.3)
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel(metric_name)
        ax.set_title(f'{metric_name.upper()} vs. Epoch')
        ax.grid(True, alpha=0.3)
        if standard_df is not None and hierarchical_df is not None:
            ax.legend()
    
    plt.tight_layout()
    
    # Save the plot
    plots_dir = os.path.join(reports_dir, "plots")
    os.makedirs(plots_dir, exist_ok=True)
    
    plot_path = os.path.join(plots_dir, f"training_metrics_{timestamp_now}.png")
    plt.savefig(plot_path)
    print(f"Saved training metrics plot to: {plot_path}")
    
    # Add plot path to training config (with JSON serialization fix)
    training_config["plots"] = training_config.get("plots", {})
    training_config["plots"]["training_metrics"] = plot_path
    
    # Fix JSON serialization issue by converting all NumPy types
    serializable_config = make_json_serializable(training_config)
    
    with open(training_config_path, 'w') as f:
        json.dump(serializable_config, f, indent=2)
    
    return plot_path

# Get potential actual paths
standard_actual_path = training_config.get('standard_actual_model_path', standard_model_path)
standard_fallback_paths = [standard_actual_path] if standard_actual_path != standard_model_path else []

hierarchical_actual_path = training_config.get('hierarchical_model', {}).get('actual_model_path', hierarchical_model_path)
hierarchical_fallback_paths = [hierarchical_actual_path] if hierarchical_actual_path != hierarchical_model_path else []

# Check for any directories that match the base names
base_standard_name = os.path.basename(standard_model_path)
base_hierarchical_name = os.path.basename(hierarchical_model_path)

print("\nLooking for model directories:")
if os.path.exists(model_save_dir):
    for dir_name in os.listdir(model_save_dir):
        dir_path = os.path.join(model_save_dir, dir_name)
        if os.path.isdir(dir_path):
            if base_standard_name in dir_name and dir_path not in standard_fallback_paths:
                print(f"Found potential standard model dir: {dir_name}")
                standard_fallback_paths.append(dir_path)
            elif base_hierarchical_name in dir_name and dir_path not in hierarchical_fallback_paths:
                print(f"Found potential hierarchical model dir: {dir_name}")
                hierarchical_fallback_paths.append(dir_path)

# Load results
print("\nLoading training results...")
standard_results_df = load_results_csv(standard_model_path, standard_fallback_paths)
hierarchical_results_df = load_results_csv(hierarchical_model_path, hierarchical_fallback_paths)

if standard_results_df is not None:
    print(f"Loaded standard model results: {len(standard_results_df)} epochs")
    print(f"Available metrics: {', '.join(standard_results_df.columns)}")
else:
    print("Standard model results not found")

if hierarchical_results_df is not None:
    print(f"Loaded hierarchical model results: {len(hierarchical_results_df)} epochs")
    print(f"Available metrics: {', '.join(hierarchical_results_df.columns)}")
else:
    print("Hierarchical model results not found")

# Plot training metrics
if standard_results_df is not None or hierarchical_results_df is not None:
    print("\nGenerating training metrics visualization...")
    plot_path = plot_training_metrics(standard_results_df, hierarchical_results_df)
    print(f"Training visualization complete: {plot_path}")
else:
    print("Cannot generate visualizations: No results data available")


Looking for model directories:
Found potential standard model dir: wildlife_detector_20250510_1706
Found potential hierarchical model dir: wildlife_detector_hierarchical_20250510_17062
Found potential hierarchical model dir: wildlife_detector_hierarchical_20250510_1706

Loading training results...
Loading results from fallback path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_20250510_17062/results.csv
Loading results from fallback path: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/models/trained/wildlife_detector_hierarchical_20250510_17062/results.csv
Loaded standard model results: 97 epochs
Available metrics: epoch, time, train/box_loss, train/cls_loss, train/dfl_loss, metrics/precision(B), metrics/recall(B), metrics/mAP50(B), metrics/mAP50-95(B), val/box_loss, val/cls_loss, val/dfl_loss, lr/pg0, lr/pg1, lr/pg2
Loaded hierarchical model results: 50 epochs
Available metrics: epoch, time, train/box_loss, train/cls_loss, train/dfl_loss, me

In [8]:
# Cell 8: Output Tracking for Long-Term Project Management
# Create detailed tracking of all generated files and models

# Create tracking directory if it doesn't exist
tracking_dir = os.path.join(project_root, "tracking")
os.makedirs(tracking_dir, exist_ok=True)

# Get actual model paths (from training config if available)
standard_actual_path = training_config.get('standard_actual_model_path', standard_model_path)
hierarchical_actual_path = training_config.get('hierarchical_model', {}).get('actual_model_path', hierarchical_model_path)

# Collect all generated files and their purposes
generated_files = {
    "configuration": {
        "training_config": training_config_path
    },
    "standard_model": {
        "base_path": standard_model_path,
        "actual_path": standard_actual_path,
        "best_weights": os.path.join(standard_actual_path, "weights", "best.pt"),
        "last_weights": os.path.join(standard_actual_path, "weights", "last.pt"),
        "results_csv": os.path.join(standard_actual_path, "results.csv"),
        "summary_report": os.path.join(reports_dir, f"standard_model_summary_{timestamp_now}.md")
    },
    "hierarchical_model": {
        "base_path": hierarchical_model_path,
        "actual_path": hierarchical_actual_path,
        "best_weights": os.path.join(hierarchical_actual_path, "weights", "best.pt"),
        "last_weights": os.path.join(hierarchical_actual_path, "weights", "last.pt"),
        "results_csv": os.path.join(hierarchical_actual_path, "results.csv"),
        "summary_report": os.path.join(reports_dir, f"hierarchical_model_summary_{timestamp_now}.md")
    },
    "visualizations": {
        "training_metrics": os.path.join(reports_dir, "plots", f"training_metrics_{timestamp_now}.png")
    }
}

# Create a comprehensive output tracking file
tracking_file = os.path.join(tracking_dir, f"notebook2_outputs_{timestamp_now}.json")
with open(tracking_file, 'w') as f:
    json.dump({
        "notebook": "02_model_training",
        "execution_timestamp": timestamp_now,
        "description": "Model training for wildlife detection",
        "generated_files": generated_files,
        "training_config": training_config,
        "next_steps": {
            "notebook": "03_model_evaluation.ipynb",
            "required_inputs": [training_config_path]
        }
    }, f, indent=2)

# Create a simple Markdown summary for human-readable reference
summary_file = os.path.join(tracking_dir, f"notebook2_summary_{timestamp_now}.md")
with open(summary_file, 'w') as f:
    f.write(f"# Model Training Notebook Outputs\n\n")
    f.write(f"**Execution Date:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")
    
    f.write(f"## Configuration\n\n")
    f.write(f"- Training Config: `{training_config_path}`\n")
    f.write(f"- Model Size: {training_config['model']['name']} ({training_config['model']['description']})\n")
    f.write(f"- Hardware: {training_config['hardware']['device']} | Batch Size: {training_config['hardware']['batch_size']} | Image Size: {training_config['hardware']['image_size']}px\n\n")
    
    # Standard model results
    if 'standard_model' in training_config and 'train_results' in training_config['standard_model']:
        std_results = training_config['standard_model']['train_results'] 
        f.write(f"## Standard Model Results\n\n")
        f.write(f"- Base Path: `{standard_model_path}`\n")
        f.write(f"- Actual Path: `{standard_actual_path}`\n")
        f.write(f"- Best Weights: `weights/best.pt`\n")
        f.write(f"- Classes: {len(class_names)}\n")
        
        # Safely get best epoch and map values
        best_epoch = std_results.get('best_epoch', 'N/A')
        maps = std_results.get('maps', [0, 0])
        map50_95 = maps[0] if len(maps) > 0 else 0
        map50 = maps[1] if len(maps) > 1 else 0
        
        f.write(f"- Best Epoch: {best_epoch}\n")
        f.write(f"- mAP50: {map50:.4f}\n")
        f.write(f"- mAP50-95: {map50_95:.4f}\n\n")
    else:
        f.write(f"## Standard Model\n\n")
        f.write(f"- Path: `{standard_model_path}`\n")
        f.write(f"- Status: Training results not available\n\n")
    
    # Hierarchical model results
    if 'hierarchical_model' in training_config and 'train_results' in training_config['hierarchical_model']:
        hier_results = training_config['hierarchical_model']['train_results']
        f.write(f"## Hierarchical Model Results\n\n")
        f.write(f"- Base Path: `{hierarchical_model_path}`\n")
        f.write(f"- Actual Path: `{hierarchical_actual_path}`\n")
        f.write(f"- Best Weights: `weights/best.pt`\n")
        f.write(f"- Taxonomic Groups: {len(taxonomic_groups)}\n")
        
        # Safely get best epoch and map values
        best_epoch = hier_results.get('best_epoch', 'N/A')
        maps = hier_results.get('maps', [0, 0])
        map50_95 = maps[0] if len(maps) > 0 else 0
        map50 = maps[1] if len(maps) > 1 else 0
        
        f.write(f"- Best Epoch: {best_epoch}\n")
        f.write(f"- mAP50: {map50:.4f}\n")
        f.write(f"- mAP50-95: {map50_95:.4f}\n\n")
    else:
        f.write(f"## Hierarchical Model\n\n")
        f.write(f"- Path: `{hierarchical_model_path}`\n")
        f.write(f"- Status: Training results not available\n\n")
    
    f.write(f"## Next Steps\n\n")
    f.write(f"Proceed to notebook 3 (Model Evaluation) using the training config: `{os.path.basename(training_config_path)}`")

print(f"\nOutput tracking files created:")
print(f"- JSON tracking: {tracking_file}")
print(f"- Markdown summary: {summary_file}")
print(f"\nThese files document all outputs from this notebook for long-term project organization.")

print("\nModel training complete!")
print("The trained models are ready for evaluation.")
print("Please proceed to the model evaluation notebook (03_model_evaluation.ipynb).")


Output tracking files created:
- JSON tracking: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/tracking/notebook2_outputs_20250510_1706.json
- Markdown summary: /home/peter/Desktop/TU PHD/WildlifeDetectionSystem/tracking/notebook2_summary_20250510_1706.md

These files document all outputs from this notebook for long-term project organization.

Model training complete!
The trained models are ready for evaluation.
Please proceed to the model evaluation notebook (03_model_evaluation.ipynb).
